# Spatial plots of tas/pr post Krakatoa eruption using CMIP6 historical model output
- opens all CMIP6 historical model run files with data on precipitation pr and surface temperature tas 
- calculates monthly anomalies (based on 1850-1880 climatology) for 4 spatial regions (global, SH, Aus, E Aus)
- generates spatial plots at 4 discrete times following Krakatoa 1883 (Aug 1883, Feb 1884, Aug 1884, Aug 1885)

In [1]:
import xarray as xr, matplotlib.pyplot as plt
from importlib import reload # need to use this if I edit a function file
import os
import numpy as np
import pandas as pd
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????
import dask.diagnostics # dask allows you to check how long something is taking to load
#import cmocean.cm as cmo # package with a nice colourbar for rainfall 

Matplotlib is building the font cache; this may take a moment.


In [2]:
# import custom functions
import sys 
# sys.path.append('/home/563/kb6999/Functions') # use this if the function file is in a different directory to the notebook
sys.path.append('/g/data/w48/kb6999/Functions') # use this if the function file is in a different directory to the notebook

import frequently_used_functions as func
import plotting_functions as fplot
import model_functions as funcM
import reanalysis_functions as funcR

In [3]:
!ls /g/data/lp01/CMIP6/CMIP/CAS/CAS-ESM2-0/historical/r1i1p1f1/Amon/pr/gr1.5

v20200302  v20201227


In [4]:
# store each section of the directory as a string
institution_dir = '/g/data/lp01/CMIP6/CMIP/'
tas_dir = '/historical/r1i1p1f1/Amon/tas/gr1.5/'
pr_dir = '/historical/r1i1p1f1/Amon/pr/gr1.5/'
print(institution_dir, tas_dir, pr_dir)

/g/data/lp01/CMIP6/CMIP/ /historical/r1i1p1f1/Amon/tas/gr1.5/ /historical/r1i1p1f1/Amon/pr/gr1.5/


## Read in model data

In [21]:
# var = 'pr'
var = 'tas'

In [6]:
if var == 'tas':
    models_tas = funcM.read_models(institution_dir, tas_dir, '1850-01','2015-01')

elif var == 'pr':
    models_pr = funcM.read_models(institution_dir, pr_dir, '1850-01','2015-01')
print('var = ', var)

53 model paths found and loaded into the dictionary "models"
53 models have been successfully loaded into an xarray
var =  tas


In [37]:
if var == 'tas':
    models = xr.Dataset({'tas': models_tas.tas})
    del models['height']
    models
elif var == 'pr':
    # combine precipitation and temperature into one dataset
    models = xr.Dataset({'pr': models_pr.pr})
    # change the pr units
    models['pr'] = models.pr*86400
    models

In [38]:
models = models.sel(model= ['ACCESS-CM2','ACCESS-ESM1-5','AWI-CM-1-1-MR','AWI-ESM-1-1-LR','BCC-CSM2-MR','BCC-ESM1',
                             'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 
                             'CMCC-CM2-HR4', 'CMCC-CM2-SR5' ,'CanESM5', 'E3SM-1-1' ,'E3SM-1-1-ECA' ,'EC-Earth3', 
                             'EC-Earth3-AerChem' , 'EC-Earth3-Veg-LR', 'FGOALS-f3-L' ,'FGOALS-g3', 'FIO-ESM-2-0', 
                             'GFDL-CM4' ,'GFDL-ESM4', 'GISS-E2-1-G' ,'GISS-E2-1-G-CC', 'GISS-E2-1-H' ,'IITM-ESM', 
                             'INM-CM4-8', 'INM-CM5-0' ,'IPSL-CM6A-LR', 'KACE-1-0-G', 'MIROC6' ,'MPI-ESM-1-2-HAM',
                             'MPI-ESM1-2-LR' ,'MRI-ESM2-0', 'NESM3', 'NorCPM1' ,'NorESM2-LM', 'NorESM2-MM',
                             'SAM0-UNICON', 'TaiESM1'])
models

,Array,Chunk
Bytes,9.13 GiB,1.32 MiB
Shape,"(43, 1980, 120, 240)","(1, 12, 120, 240)"
Count,33383 Tasks,7095 Chunks
Type,float32,numpy.ndarray


In [39]:
# sort models alphabetically and prints final model list
models = models.sortby('model')
print('The following', len(models.model.data), 'models will be used in all subsequent calculations: \n')
print(models.model.data)

The following 43 models will be used in all subsequent calculations: 

['ACCESS-CM2' 'ACCESS-ESM1-5' 'AWI-CM-1-1-MR' 'AWI-ESM-1-1-LR'
 'BCC-CSM2-MR' 'BCC-ESM1' 'CAMS-CSM1-0' 'CAS-ESM2-0' 'CESM2' 'CESM2-FV2'
 'CESM2-WACCM' 'CESM2-WACCM-FV2' 'CMCC-CM2-HR4' 'CMCC-CM2-SR5' 'CanESM5'
 'E3SM-1-1' 'E3SM-1-1-ECA' 'EC-Earth3' 'EC-Earth3-AerChem'
 'EC-Earth3-Veg-LR' 'FGOALS-f3-L' 'FGOALS-g3' 'FIO-ESM-2-0' 'GFDL-CM4'
 'GFDL-ESM4' 'GISS-E2-1-G' 'GISS-E2-1-G-CC' 'GISS-E2-1-H' 'IITM-ESM'
 'INM-CM4-8' 'INM-CM5-0' 'IPSL-CM6A-LR' 'KACE-1-0-G' 'MIROC6'
 'MPI-ESM-1-2-HAM' 'MPI-ESM1-2-LR' 'MRI-ESM2-0' 'NESM3' 'NorCPM1'
 'NorESM2-LM' 'NorESM2-MM' 'SAM0-UNICON' 'TaiESM1']


## Select out regions
Right now I have all the models stored in one array so from hereafter I can calculate anomalies etc.  

In [40]:
# area weighting 
models_w = models*np.cos(models.lat*(np.pi/180))

In [41]:
# import land fraction data
landfrac_ds = xr.open_dataset('/g/data/w48/kb6999/Old_Models/landfraction_file_grid1.5.nc')
landmask = landfrac_ds.mean(dim='time')

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/xarray/coding/times.py:119: SerializationWarning: Ambiguous reference date string: 1-01-01 01:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-01-01 01:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/xarray/coding/times.py:527: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/xarray/core/indexing.py:572: SerializationWarning: Unable to decode time 

## Anomalies

In [42]:
# use functions to calculate the monthly anomalies for the globe 
monthly_anom_glob = func.monthly_anomaly(models_w, '1850-01', '1881-01')
monthly_anom_glob

,Array,Chunk
Bytes,18.27 GiB,225.00 kiB
Shape,"(43, 1980, 120, 240)","(1, 1, 120, 240)"
Count,422749 Tasks,85140 Chunks
Type,float64,numpy.ndarray


In [43]:
# select out regions for other anomalies 
monthly_anom_SH = monthly_anom_glob.sel(lat=slice(-90,0)) 
monthly_anom_NH = monthly_anom_glob.sel(lat=slice(0,90)) 
monthly_anom_Aus = monthly_anom_glob.sel(lat=slice(-50,-10), lon=slice(110,160)).where(landmask.data==1, drop=True)
monthly_anom_EA = monthly_anom_glob.sel(lat=slice(-50,-10), lon=slice(140,155)).where(landmask.data==1, drop=True)

In [44]:
monthly_anom_EA

,Array,Chunk
Bytes,122.77 MiB,1.48 kiB
Shape,"(43, 1980, 21, 9)","(1, 1, 21, 9)"
Count,763310 Tasks,85140 Chunks
Type,float64,numpy.ndarray


In [45]:
# take lat lon mean
llm_Glob = monthly_anom_glob.mean(dim=['lat','lon'])
llm_SH = monthly_anom_SH.mean(dim=['lat','lon'])
llm_NH = monthly_anom_NH.mean(dim=['lat','lon'])
llm_Aus = monthly_anom_Aus.mean(dim=['lat','lon'])
llm_EA = monthly_anom_EA.mean(dim=['lat','lon'])

In [46]:
# Take the multimodel mean for each region
mmm_mon_Glob = monthly_anom_glob.mean(dim='model').mean(dim=['lat','lon'])
mmm_mon_SH = monthly_anom_SH.mean(dim='model').mean(dim=['lat','lon'])
mmm_mon_NH = monthly_anom_NH.mean(dim='model').mean(dim=['lat','lon'])
mmm_mon_Aus = monthly_anom_Aus.mean(dim='model').mean(dim=['lat','lon'])
mmm_mon_EA = monthly_anom_EA.mean(dim='model').mean(dim=['lat','lon'])
mmm_mon_EA

<xarray.Dataset>
Dimensions:  (time: 1980)
Coordinates:
  * time     (time) datetime64[ns] 1850-01-31 1850-02-28 ... 2014-12-31
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    tas      (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>

In [47]:
# eruption date
e_date = pd.to_datetime(['1883-08','1902-10', '1963-03', '1982-04', '1991-06'])
# save arrays of times relative to each eruption
e_pre5yrs = e_date - 60 * pd.offsets.MonthBegin()
e_pre3mon = e_date - 3 * pd.offsets.MonthBegin()
e_post5yrs = e_date + 61 * pd.offsets.MonthBegin()
e_post10yrs = e_date + 121 * pd.offsets.MonthBegin()

In [48]:
# Zoom in on Krakatoa time period
K_mmm_Glob = mmm_mon_Glob.sel(time = slice(e_pre5yrs[0], e_post5yrs[0]))
K_mmm_SH = mmm_mon_SH.sel(time = slice(e_pre5yrs[0], e_post5yrs[0]))
K_mmm_Aus = mmm_mon_Aus.sel(time = slice(e_pre5yrs[0], e_post5yrs[0]))
K_mmm_EA = mmm_mon_EA.sel(time = slice(e_pre5yrs[0], e_post5yrs[0]))

In [49]:
# calculate the percentiles and then take the mean along the model dimension for monthly anomalies
p10_mon_Glob = llm_Glob.chunk({'model': -1}).quantile(0.1, dim=['model'])
p10_mon_SH = llm_SH.chunk({'model': -1}).quantile(0.1, dim=['model'])
p10_mon_NH = llm_NH.chunk({'model': -1}).quantile(0.1, dim=['model'])
p10_mon_Aus = llm_Aus.chunk({'model': -1}).quantile(0.1, dim=['model'])
p10_mon_EA = llm_EA.chunk({'model': -1}).quantile(0.1, dim=['model'])

In [50]:
# calculate the percentiles and then take the mean along the model dimension for monthly anomalies
p90_mon_Glob = llm_Glob.chunk({'model': -1}).quantile(0.9, dim=['model'])
p90_mon_SH = llm_SH.chunk({'model': -1}).quantile(0.9, dim=['model'])
p90_mon_NH = llm_NH.chunk({'model': -1}).quantile(0.9, dim=['model'])
p90_mon_Aus = llm_Aus.chunk({'model': -1}).quantile(0.9, dim=['model'])
p90_mon_EA = llm_EA.chunk({'model': -1}).quantile(0.9, dim=['model'])
p90_mon_EA

<xarray.Dataset>
Dimensions:   (time: 1980)
Coordinates:
  * time      (time) datetime64[ns] 1850-01-31 1850-02-28 ... 2014-12-31
    month     (time) int64 dask.array<chunksize=(1980,), meta=np.ndarray>
    quantile  float64 0.9
Data variables:
    tas       (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>

### significance 

In [51]:
# define array with region name, and then corresponding time series 
regions = ['Glob','SH','Aus','EA']
if var == 'pr':
    time_series = [mmm_mon_Glob.pr, mmm_mon_SH.pr, mmm_mon_Aus.pr, mmm_mon_EA.pr]
elif var == 'tas':
    time_series = [mmm_mon_Glob.tas, mmm_mon_SH.tas, mmm_mon_Aus.tas, mmm_mon_EA.tas]

data = {r: d for r, d in zip(regions, time_series)}

In [20]:
# calculate and print statistical significance of 2 standard deviations
if var == 'pr':
    func.sig_2std_vals(data, var='precipitation')

Significance of 2 standard deviations of precipitation signal on each spatial scale is:
Glob: 0.006 mm/day
SH: 0.018 mm/day


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


Aus: 0.189 mm/day


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


EA: 0.25 mm/day


In [36]:
if var == 'tas':
    func.sig_2std_vals(data, var='temperature')

Significance of 2 standard deviations of temperature signal on each spatial scale is:
Glob: 0.01 °C
SH: 0.02 °C


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


Aus: 0.19 °C


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


EA: 0.25 °C


In [25]:
func=reload(func)

### maximum

In [56]:
K_mmm_Glob_max = K_mmm_Glob.max()
K_mmm_SH_max = K_mmm_SH.max()
K_mmm_Aus_max = K_mmm_Aus.max()
K_mmm_EA_max = K_mmm_EA.max()
K_mmm_EA_max

<xarray.Dataset>
Dimensions:  ()
Data variables:
    tas      float64 dask.array<chunksize=(), meta=np.ndarray>

In [ ]:
K_mmm_EA_max.tas.values

In [ ]:
 # find the index of the minimum anomaly
    min_index = anom_dataset.where(anom_dataset == min_dataset).argmin('time')
    
    # These indices can be converted to time values all at once by using min_index as an array index
    # find the date the minimum value occured for both tas and pr and remove time axis (using drop)
    # (There's a "ghost" time axis in the result which might cause problems, let's get rid of it.)
    # (Our returned values aren't time dependent)
    min_date = anom_dataset.time[min_index].drop('time')

In [52]:
# maximum values 
# calculate the minimum value of the mmm for each spatial region

max_date_Glob = func.min_date(K_mmm_Glob.tas, K_mmm_Glob_max.tas).dt.strftime("%b %Y")
max_date_SH = func.min_date(K_mmm_SH.tas, K_mmm_SH_max.tas).dt.strftime("%b %Y")
max_date_Aus = func.min_date(K_mmm_Aus.tas, K_mmm_Aus_max.tas).dt.strftime("%b %Y")
max_date_EA = func.min_date(K_mmm_EA.tas, K_mmm_EA_max.tas).dt.strftime("%b %Y")

AttributeError: 'Array' object has no attribute 'item'

In [35]:
# minimum values 
# calculate the minimum value of the mmm for each spatial region
K_mmm_Glob_min = K_mmm_Glob.min()
K_mmm_SH_min = K_mmm_SH.min()
K_mmm_Aus_min = K_mmm_Aus.min()
K_mmm_EA_min = K_mmm_EA.min()
max_date_Glob = func.min_date(K_mmm_Glob.tas, K_mmm_Glob_min.tas).dt.strftime("%b %Y")
max_date_SH = func.min_date(K_mmm_SH.tas, K_mmm_SH_min.tas).dt.strftime("%b %Y")
max_date_Aus = func.min_date(K_mmm_Aus.tas, K_mmm_Aus_min.tas).dt.strftime("%b %Y")
max_date_EA = func.min_date(K_mmm_EA.tas, K_mmm_EA_min.tas).dt.strftime("%b %Y")

AttributeError: 'Array' object has no attribute 'item'

In [54]:
if var == 'tas':
    func.max_vals(data, var='temperature')

Maximum value of roll12 precipitation signal on each spatial scale is:


AttributeError: 'Array' object has no attribute 'item'

In [ ]:
if var == 'pr':
    func.max_vals(data, var='temperature')

## To netcdf

In [19]:
if var == 'tas':
    path = '/g/data/w48/kb6999/Models/models_tas_data'
if var == 'pr':
    path = '/g/data/w48/kb6999/Models/models_pr_data'

In [ ]:
# mmm and llm
with dask.diagnostics.ProgressBar():
#     mmm_mon_Glob.to_netcdf(f'{path}mmm_mon_Glob.nc')
#     mmm_mon_NH.to_netcdf(f'{path}mmm_mon_NH.nc')
#     mmm_mon_SH.to_netcdf(f'{path}mmm_mon_SH.nc')
    mmm_mon_Aus.to_netcdf(f'{path}mmm_mon_Aus.nc')
#     mmm_mon_EA.to_netcdf(f'{path}mmm_mon_EA.nc')

[                                        ] | 1% Completed |  1min 51.8s

In [ ]:
with dask.diagnostics.ProgressBar():
    p10_mon_Glob.to_netcdf(f'{path}p10_mon_Glob.nc')
    p10_mon_NH.to_netcdf(f'{path}p10_mon_NH.nc')
    p10_mon_SH.to_netcdf(f'{path}p10_mon_SH.nc')
    p10_mon_Aus.to_netcdf(f'{path}p10_mon_Aus.nc')
    p10_mon_EA.to_netcdf(f'{path}p10_mon_EA.nc')

In [ ]:
with dask.diagnostics.ProgressBar():
    p90_mon_Glob.to_netcdf(f'{path}p90_mon_Glob.nc')
    p90_mon_NH.to_netcdf(f'{path}p90_mon_NH.nc')
    p90_mon_SH.to_netcdf(f'{path}p90_mon_SH.nc')
    p90_mon_Aus.to_netcdf(f'{path}p90_mon_Aus.nc')
    p90_mon_EA.to_netcdf(f'{path}p90_mon_EA.nc')

In [ ]:
with dask.diagnostics.ProgressBar():
    llm_Glob.to_netcdf(f'{path}llm_mon_Glob.nc')
    llm_SH.to_netcdf(f'{path}llm_mon_NH.nc') 
    llm_NH.to_netcdf(f'{path}llm_mon_SH.nc')
    llm_Aus.to_netcdf(f'{path}llm_mon_Aus.nc')
    llm_EA.to_netcdf(f'{path}llm_mon_EA.nc')

[                                        ] | 2% Completed |  6min 37.7s